In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from google.colab import userdata

In [ ]:
app = FastAPI()

In [ ]:
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTHTOKEN")

In [ ]:
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins = origins,
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"]
)

In [ ]:
class ModelInput(BaseModel):
    Pregnancies              : int
    Glucose                  : int
    BloodPressure            : int
    SkinThickness            : int
    Insulin                  : int
    BMI                      : float
    DiabetesPedigreeFunction : float
    Age                      : int

In [ ]:
diabetes_model = pickle.load(open("/content/diabetes_model.sav", "rb"))

In [ ]:
@app.post('/diabetes-prediction')
def diabetes_pred(input_parameters: ModelInput):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    preg   = input_dictionary['Pregnancies']
    gluc   = input_dictionary['Glucose']
    bp     = input_dictionary['BloodPressure']
    skinth = input_dictionary['SkinThickness']
    ins    = input_dictionary['Insulin']
    bmi    = input_dictionary['BMI']
    dpf    = input_dictionary['DiabetesPedigreeFunction']
    age    = input_dictionary['Age']

    input_list = [preg, gluc, bp, skinth, ins, bmi, dpf, age]
    prediction = diabetes_model.predict([input_list])

    if prediction[0] == 0:
        return "The person does not have Diabetes"
    else:
        return "The person has Diabetes"

In [ ]:
nest_asyncio.apply()
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()

Public URL: https://barbara-unconspired-thusly.ngrok-free.dev


INFO:     Started server process [171]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.121.71.1:0 - "POST /diabetes-prediction HTTP/1.1" 200 OK


/tmp/ipython-input-3297110290.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  input_data = input_parameters.json()
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
<frozen abc>:123: RuntimeWarning: coroutine 'Server.serve' was never awaited


INFO:     103.121.71.1:0 - "POST /diabetes-prediction HTTP/1.1" 200 OK


/tmp/ipython-input-3297110290.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  input_data = input_parameters.json()
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [171]
